In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Print GPU details
    for gpu in gpus:
        print("Name:", gpu.name, "Type:", gpu.device_type)
    


# Verify TensorFlow using GPU
print("Is built with CUDA:", tf.test.is_built_with_cuda())
print("Is GPU available:", tf.test.is_gpu_available())

2025-03-20 15:03:02.470076: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-20 15:03:02.548180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742482982.591309    5410 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742482982.601425    5410 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742482982.675978    5410 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Num GPUs Available:  1
Name: /physical_device:GPU:0 Type: GPU
Is built with CUDA: True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: True


I0000 00:00:1742482984.238299    5410 gpu_device.cc:2019] Created device /device:GPU:0 with 9750 MB memory:  -> device: 0, name: NVIDIA RTX 3500 Ada Generation Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:

#np.random.seed(42)
#tf.random.set_seed(42)

In [ ]:
# Data directories paths
base_dir = '/home/kacper/Y4S2/DataScience/DataScienceSem2/archive (2)/ASL_Alphabet_Dataset'
train_dir = os.path.join(base_dir, 'asl_alphabet_train')
test_dir = os.path.join(base_dir, 'asl_alphabet_test')


In [ ]:
# Image parameters
img_width, img_height = 128, 128 #higher resolution
batch_size = 256 

In [ ]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Using 20% of training data for validation
)

In [5]:
# Only rescale for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data with augmentation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 178472 images belonging to 29 classes.


In [6]:
# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 44602 images belonging to 29 classes.


In [7]:
# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 0 images belonging to 0 classes.


In [8]:
# Get number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")
print("Class mapping:", train_generator.class_indices)

Number of classes: 29
Class mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


In [9]:
# Build CNN model
model = Sequential([
    # First convolutional block
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_width, img_height, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Second convolutional block
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Third convolutional block
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Fully connected layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

/home/kacper/Y4S2/DataScience/DataScienceSem2/.venv/lib64/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1742483015.137654    5410 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9750 MB memory:  -> device: 0, name: NVIDIA RTX 3500 Ada Generation Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [10]:
# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 126, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 63, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 61, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,147,453 (50.15 MB)

 Trainable params: 13,147,453 (50.15 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6
)

In [ ]:
#%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:

epochs = 10

# For my available 12.8GB VRAM, may need to lower
batch_size = 256  

# Enable mixed precision if available (allows larger batches from tensorflow package)
if hasattr(tf.keras.mixed_precision, 'set_global_policy'):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled - can use larger batch sizes")

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

Mixed precision enabled - can use larger batch sizes


/home/kacper/Y4S2/DataScience/DataScienceSem2/.venv/lib64/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1742483028.268044    5964 service.cc:152] XLA service 0x7f2050003e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742483028.268059    5964 service.cc:160]   StreamExecutor device (0): NVIDIA RTX 3500 Ada Generation Laptop GPU, Compute Capability 8.9
2025-03-20 15:03:48.293050: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742483028.475521    5964 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-20 15:03:49.320486: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2155', 64 bytes spill stores, 64 bytes spill loads

2025-03-20 15:03:49.366284: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gem

  1/697 ━━━━━━━━━━━━━━━━━━━━ 4:15:36 22s/step - accuracy: 0.0430 - loss: 3.3558

...

697/697 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step - accuracy: 0.1334 - loss: 2.9945

/home/kacper/Y4S2/DataScience/DataScienceSem2/.venv/lib64/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-03-20 15:09:34.224745: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_138_0', 136 bytes spill stores, 136 bytes spill loads

2025-03-20 15:09:34.358009: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_138', 48 bytes spill stores, 48 bytes spill loads

2025-03-20 15:09:34.382288: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346

697/697 ━━━━━━━━━━━━━━━━━━━━ 458s 626ms/step - accuracy: 0.1336 - loss: 2.9937 - val_accuracy: 0.5683 - val_loss: 1.5488 - learning_rate: 0.0010
Epoch 2/10
  1/697 ━━━━━━━━━━━━━━━━━━━━ 1:35 137ms/step - accuracy: 0.5469 - loss: 1.4030

/home/kacper/Y4S2/DataScience/DataScienceSem2/.venv/lib64/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


697/697 ━━━━━━━━━━━━━━━━━━━━ 92s 132ms/step - accuracy: 0.5469 - loss: 1.4030 - val_accuracy: 0.5685 - val_loss: 1.5416 - learning_rate: 0.0010
Epoch 3/10
697/697 ━━━━━━━━━━━━━━━━━━━━ 364s 522ms/step - accuracy: 0.6264 - loss: 1.1317 - val_accuracy: 0.6873 - val_loss: 1.2656 - learning_rate: 0.0010
Epoch 4/10
697/697 ━━━━━━━━━━━━━━━━━━━━ 91s 131ms/step - accuracy: 0.6680 - loss: 0.8491 - val_accuracy: 0.6922 - val_loss: 1.3046 - learning_rate: 0.0010
Epoch 5/10
697/697 ━━━━━━━━━━━━━━━━━━━━ 365s 523ms/step - accuracy: 0.7642 - loss: 0.7041 - val_accuracy: 0.7449 - val_loss: 1.0068 - learning_rate: 0.0010
Epoch 6/10
697/697 ━━━━━━━━━━━━━━━━━━━━ 89s 128ms/step - accuracy: 0.7734 - loss: 0.6571 - val_accuracy: 0.7451 - val_loss: 1.0106 - learning_rate: 0.0010
Epoch 7/10
697/697 ━━━━━━━━━━━━━━━━━━━━ 364s 522ms/step - accuracy: 0.8185 - loss: 0.5392 - val_accuracy: 0.7675 - val_loss: 1.0879 - learning_rate: 0.0010
Epoch 8/10
697/697 ━━━━━━━━━━━━━━━━━━━━ 91s 131ms/step - accuracy: 0.8359 - lo